In [3]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /usr/local/lib/python3.7/dist-packages (0.0)


In [4]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [36]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [38]:
from google.colab import files
uploaded = files.upload()

Saving beer_data_set.csv to beer_data_set (1).csv


In [41]:

import io
df = pd.read_csv(io.BytesIO(uploaded['beer_data_set.csv']))
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,Name,key,Style,Style Key,Brewery,Description,ABV,Ave Rating,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,13,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,12,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,14,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,13,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,21,69,10,63,120,14,0,19,36,15,218


In [42]:
df.columns

Index(['Name', 'key', 'Style', 'Style Key', 'Brewery', 'Description', 'ABV',
       'Ave Rating', 'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol',
       'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices',
       'Malty'],
      dtype='object')

In [43]:
# df = df.groupby(['beer_name'])
# df


In [44]:
# Name = df['beer_name'].first()
# Name

# Select your features (columns)

In [71]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]
X = df.drop(['Name', 'key', 'Style', 'Style Key', 'Brewery', 'Description'],axis=1)
y = df['Style']
print(X.shape, y.shape)


(5556, 15) (5556,)


# Create a Train Test Split



In [72]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [73]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [74]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [75]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [76]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [82]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=15))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=112, activation='softmax'))

In [83]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [84]:
import numpy as np
y_train = np.asarray(y_train)
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
131/131 - 1s - loss: 4.5557 - accuracy: 0.0398
Epoch 2/100
131/131 - 0s - loss: 3.5684 - accuracy: 0.1560
Epoch 3/100
131/131 - 0s - loss: 2.7187 - accuracy: 0.3026
Epoch 4/100
131/131 - 0s - loss: 2.3045 - accuracy: 0.3837
Epoch 5/100
131/131 - 0s - loss: 2.0428 - accuracy: 0.4341
Epoch 6/100
131/131 - 0s - loss: 1.8525 - accuracy: 0.4795
Epoch 7/100
131/131 - 0s - loss: 1.7035 - accuracy: 0.5128
Epoch 8/100
131/131 - 0s - loss: 1.5653 - accuracy: 0.5515
Epoch 9/100
131/131 - 0s - loss: 1.4562 - accuracy: 0.5846
Epoch 10/100
131/131 - 0s - loss: 1.3603 - accuracy: 0.6002
Epoch 11/100
131/131 - 0s - loss: 1.2688 - accuracy: 0.6359
Epoch 12/100
131/131 - 0s - loss: 1.2046 - accuracy: 0.6422
Epoch 13/100
131/131 - 0s - loss: 1.1363 - accuracy: 0.6652
Epoch 14/100
131/131 - 0s - loss: 1.0743 - accuracy: 0.6823
Epoch 15/100
131/131 - 0s - loss: 1.0110 - accuracy: 0.7082
Epoch 16/100
131/131 - 0s - loss: 0.9640 - accuracy: 0.7221
Epoch 17/100
131/131 - 0s - loss: 0.9232 - accura

In [85]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               1600      
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 112)               11312     
Total params: 23,012
Trainable params: 23,012
Non-trainable params: 0
_________________________________________________________________


In [86]:
# Save the model
model.save("neural_networks_model.h5")

In [87]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("neural_networks_model.h5")

In [88]:
# Evaluate our trained model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

44/44 - 0s - loss: 0.6108 - accuracy: 0.8186
Normal Neural Network - Loss: 0.6107589602470398, Accuracy: 0.8185744881629944


In [89]:
# make predictions
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [90]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['Lambic - Fruit' 'Farmhouse Ale - Bière de Garde' 'Stout - American'
 'Farmhouse Ale - Sahti' 'Brett Beer' 'Lager - Japanese Rice'
 'Lager - Helles' 'Stout - American Imperial' 'Scottish Ale'
 'Blonde Ale - Belgian']
Actual Labels: ['Lambic - Fruit', 'Farmhouse Ale - Bière de Garde', 'Stout - American', 'Gruit / Ancient Herbed Ale', 'Brett Beer', 'Lager - Japanese Rice', 'Lager - Helles', 'Stout - American Imperial', 'Scottish Ale', 'Blonde Ale - Belgian']
